# Orbit extrapolation notebook

In [1]:
from fds.config import set_url, set_api_key
set_url("https://api.spacetower.exotrail.space/fds/v1")
set_api_key('MY_API_KEY')

In [24]:
import datetime

import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots


from fds.models.ground_station import GroundStation
from fds.models.orbit_extrapolation.requests import OemRequest, EventsRequestStationVisibility, EventsRequestOrbital, \
    MeasurementsRequestGpsNmea, EphemeridesRequest
from fds.models.orbit_extrapolation.use_case import OrbitExtrapolation
from fds.models.orbital_state import PropagationContext, CovarianceMatrix, OrbitalState
from fds.models.spacecraft import Battery, SolarArray, ThrusterElectrical, SpacecraftBox
from fds.models.two_line_element import TwoLineElement
from fds.utils.frames import Frame

In [13]:
object_id = 25544 # NoradID
object_name = "ISS"

tle_iss = TwoLineElement("1 25544U 98067A   24142.35003124  .00022843  00000-0  38371-3 0  9995",
"2 25544  51.6390  88.3709 0003333 191.4959 306.2513 15.51667899454382")

print(f"Latest TLE: {tle_iss.single_line}")
print(f"Latest TLE date (UTC): {tle_iss.date}")
print()

Latest TLE: 1 25544U 98067A   24142.35003124  .00022843  00000-0  38371-3 0  9995
2 25544  51.6390  88.3709 0003333 191.4959 306.2513 15.51667899454382
Latest TLE date (UTC): 2024-05-21 08:24:02.699136+00:00



## Create FDS models

In [47]:
propagation_context = PropagationContext(
    model_perturbations=[
        PropagationContext.Perturbation.DRAG,
        PropagationContext.Perturbation.EARTH_POTENTIAL,
        PropagationContext.Perturbation.SRP,
        PropagationContext.Perturbation.THIRD_BODY,
    ],
    model_solar_flux=150,  # SFU
    model_earth_potential_deg=30,
    model_earth_potential_ord=30,
    model_atmosphere_kind=PropagationContext.AtmosphereModel.HARRIS_PRIESTER,
    integrator_kind=PropagationContext.IntegratorKind.DORMAND_PRINCE_853,
    integrator_min_step=0.01,  # s
    integrator_max_step=100,  # s
)

battery = Battery(
    depth_of_discharge=0.3,  # 0<x<1
    nominal_capacity=560,  # W
    minimum_charge_for_firing=0.9,  # 0<x<1
    initial_charge=1,  # 0<x<1
)

solar_array = SolarArray(
    kind="DEPLOYABLE_FIXED",
    initialisation_kind=SolarArray.InitialisationKind.MAXIMUM_POWER,
    efficiency=.293,  # 0<x<1
    normal_in_satellite_frame=(0, 0, -1),  # Unit vector
    maximum_power=300,  # W
    surface=0.749,  # m^2
)

electrical_thruster = ThrusterElectrical(
    isp=950,  # s
    thrust=0.005,  # N
    axis_in_satellite_frame=(-1, 0, 0),  # Unit vector
    propellant_mass=4,  # kg
    wet_mass=11,  # kg
    warm_up_duration=240,  # s
    maximum_thrust_duration=1200,  # s
    impulse=37265.27,  # Ns
    power=300,  # W
    stand_by_power=1.1,  # W
    warm_up_power=50,  # W
)

spacecraft = SpacecraftBox(
    battery=battery,
    thruster=electrical_thruster,
    solar_array=solar_array,
    platform_mass=112,  # kg
    drag_coefficient=2.2,
    length_x=.5,  # m
    length_y=.5,  # m
    length_z=.5,  # m
    max_angular_velocity=2,  # deg/s
    max_angular_acceleration=.5,  # deg/s^2
)

covariance = CovarianceMatrix.from_diagonal(
    diagonal=(100, 100, 100, 0.1, 0.1, 0.1),
    frame=Frame.TNW
)

# Generate initial OrbitalState
orbital_state = OrbitalState.from_tle(
    tle=tle_iss,
    covariance_matrix=covariance,
    propagation_context=propagation_context,
    spacecraft=spacecraft,
)

# Orbit Data Message request
oem_request = OemRequest(
    creator="User",
    object_id=str(object_id),
    object_name=object_name,
    frame=Frame.EME2000,
    write_acceleration=False,
    ephemerides_step=60,
)

# Events requests
station_events_request = EventsRequestStationVisibility(
    start_date=orbital_state.date,
    ground_stations=[GroundStation("Toulouse", 43.6047, 1.4442, 0.115, 5, )]
)
orbital_events = EventsRequestOrbital(
    event_kinds=[EventsRequestOrbital.EventKind.ECLIPSE,
                 EventsRequestOrbital.EventKind.NODE],
    start_date=orbital_state.date)

# Measurement request
gps_nmea_request = MeasurementsRequestGpsNmea(
    standard_deviation_altitude=1E-10,
    standard_deviation_latitude=1E-10,
    standard_deviation_longitude=1E-10,
    standard_deviation_ground_speed=1E-10,
    generation_step=60
)

ephemerides_request = EphemeridesRequest(
    ephemeris_types=[EphemeridesRequest.EphemerisType.KEPLERIAN],
    step=200,
)



## Perform the use case

In [48]:
target_date = orbital_state.date + datetime.timedelta(
    seconds=orbital_state.mean_orbit.keplerian_period * 3)

oe = OrbitExtrapolation.with_target_date(
    target_date=target_date,
    initial_orbital_state=orbital_state,
    orbit_data_message_request=oem_request,
    measurements_request=gps_nmea_request,
    orbital_events_request=orbital_events,
    station_visibility_events_request=station_events_request,
    ephemerides_request=ephemerides_request
)
print("Propagation orbit from TLE date to 3 orbits from now")
print(f"Orbit extrapolation start date: {oe.initial_date}")
print(f"Orbit extrapolation end date: {oe.final_date}")
print(f"Duration: {format(oe.duration / 3600., ".3f")} hours")

Propagation orbit from TLE date to 3 orbits from now
Orbit extrapolation start date: 2024-05-21 08:24:02.699136+00:00
Orbit extrapolation end date: 2024-05-21 13:02:29.173077+00:00
Duration: 4.641 hours


In [49]:
res = oe.run().result

### Plotting the results

In [50]:
dates = np.array(res.computed_measurements[0].dates)
latitude = np.array(res.computed_measurements[0].measurements)[:, 0]
longitude = np.array(res.computed_measurements[0].measurements)[:, 1]

# cut elements before now
now = datetime.datetime.now(datetime.UTC)
now_index = np.argmax(dates > now)
dates = dates[now_index:]
latitude = latitude[now_index:]
longitude = longitude[now_index:]

relative_times = [(date - dates[0]).total_seconds() / 3600 for date in dates]

# find time closest to station visibility start and end
station_visibility_start = []
station_visibility_end = []
lat_closest_start, lon_closest_start, lat_closest_end, lon_closest_end = [], [], [], []
if res.station_visibility_events is not None:
    for event in res.station_visibility_events:
        # check if event is before now
        if event.start_date < now:
            continue
        station_visibility_start.append(np.argmin(np.abs(dates - event.start_date)))
        station_visibility_end.append(np.argmin(np.abs(dates - event.end_date)))

    lat_closest_start = latitude[station_visibility_start]
    lon_closest_start = longitude[station_visibility_start]
    lat_closest_end = latitude[station_visibility_end]
    lon_closest_end = longitude[station_visibility_end]

lon_lat_df = pd.DataFrame({'Longitude': longitude, 'Latitude': latitude, 'Date': dates})

fig = go.Figure()
fig.add_trace(go.Scattergeo(
    lat=lon_lat_df['Latitude'],
    lon=lon_lat_df['Longitude'],
    mode='markers+lines',
    line=dict(width=1, color='black'),
    marker=dict(
        size=4,
        color='black',
    ),
    name="Orbit track",
))

# Add a point for the initial position
fig.add_trace(go.Scattergeo(
    lat=[latitude[0]],
    lon=[longitude[0]],
    mode='markers',
    marker=dict(
        size=8,
        color='red',
    ),
    name=f"Initial position at {dates[0]}",
))

# Add a point for the ground station closest to the start of the visibility
if res.station_visibility_events is not None:
    for i in range(len(station_visibility_start)):
        fig.add_trace(go.Scattergeo(
            lat=[lat_closest_start[i]],
            lon=[lon_closest_start[i]],
            mode='markers',
            marker=dict(
                size=12,
                color='green',
                symbol='triangle-up'
            ),
            name=f"Visibility start ({dates[station_visibility_start[i]]})",
        ))

    # Add a point for the ground station closest to the end of the visibility
    for i in range(len(station_visibility_end)):
        fig.add_trace(go.Scattergeo(
            lat=[lat_closest_end[i]],
            lon=[lon_closest_end[i]],
            mode='markers',
            marker=dict(
                size=12,
                color='orange',
                symbol='triangle-down'
            ),
            name=f"Visibility end ({dates[station_visibility_end[i]]})",
        ))

# Add a point for the target position
fig.add_trace(go.Scattergeo(
    lat=[station_events_request.ground_stations[0].coordinates.latitude],
    lon=[station_events_request.ground_stations[0].coordinates.longitude],
    mode='markers',
    marker=dict(
        size=8,
        color='black',
    ),
    name=station_events_request.ground_stations[0].name,
))

fig.update_layout(
    title_text=f"Orbit track",
)
fig.show()

In [11]:
data = res.export_event_timeline_data()
df = pd.DataFrame(data)
df

,date,event,ground_station_name,ground_station_id
0,2024-05-21 08:34:54.390493+00:00,DESCENDING_NODE,,
1,2024-05-21 08:51:13.816232+00:00,ECLIPSE_EXIT,,
2,2024-05-21 09:21:19.567817+00:00,ASCENDING_NODE,,
3,2024-05-21 09:50:48.674247+00:00,ECLIPSE_ENTER,,
4,2024-05-21 10:07:39.061613+00:00,DESCENDING_NODE,,
5,2024-05-21 10:24:03.980059+00:00,ECLIPSE_EXIT,,
6,2024-05-21 10:54:04.180100+00:00,ASCENDING_NODE,,
7,2024-05-21 11:23:35.727915+00:00,ECLIPSE_ENTER,,
8,2024-05-21 11:40:23.690694+00:00,DESCENDING_NODE,,
9,2024-05-21 11:56:54.065266+00:00,ECLIPSE_EXIT,,


In [51]:
dat = res.export_event_gantt_data()
df_timeline = pd.DataFrame(dat)
# Filter out DESCENDING and ASCENDING events
df_timeline = df_timeline[df_timeline["event"] != "DESCENDING_NODE"]
df_timeline = df_timeline[df_timeline["event"] != "ASCENDING_NODE"]
fig = px.timeline(df_timeline, x_start="start_date", x_end="end_date", y="event", color="event",
                  labels={'Task': 'Event'},
                  title='Events timeline',
                  hover_name="event",
                  hover_data={"ground_station_name": True}
                  )
fig.show()

# %% Extract the ephemerides
ephemerides = res.export_keplerian_ephemeris()

eph_df = pd.DataFrame(ephemerides)

# plot
fig = make_subplots(
    rows=3, cols=1,
    subplot_titles=("Semi-major axis [km]", "Eccentricity [-]", "Inclination [deg]"),
)
# Add traces to the subplots
fig.add_trace(go.Scatter(x=eph_df['date'], y=eph_df['semi_major_axis'], mode='lines', name='a'), row=1, col=1)
fig.add_trace(go.Scatter(x=eph_df['date'], y=eph_df['eccentricity'], mode='lines', name='e'), row=2, col=1)
fig.add_trace(go.Scatter(x=eph_df['date'], y=np.degrees(eph_df['inclination']), mode='lines', name='i'), row=3, col=1)
# Update layout
fig.update_layout(
    title_text="Osculating Keplerian elements"
)

fig.show()

                                  0
0  2024-05-21 08:24:02.699136+00:00
1  2024-05-21 08:25:02.699136+00:00
2  2024-05-21 08:26:02.699136+00:00
3  2024-05-21 08:27:02.699136+00:00
4  2024-05-21 08:28:02.699136+00:00
5  2024-05-21 08:29:02.699136+00:00
6  2024-05-21 08:30:02.699136+00:00
7  2024-05-21 08:31:02.699136+00:00
8  2024-05-21 08:32:02.699136+00:00
9  2024-05-21 08:33:02.699136+00:00
10 2024-05-21 08:34:02.699136+00:00
11 2024-05-21 08:35:02.699136+00:00
12 2024-05-21 08:36:02.699136+00:00
13 2024-05-21 08:37:02.699136+00:00
14 2024-05-21 08:38:02.699136+00:00
